# ShallowNet

Although car make classification is a complex task with over 196 classes. we started with a Shallownet to establish a baseline and get a sense of how convolution detects features.



In [8]:
from CVPreClass import CVPreClass as cvpre

In [9]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

In [10]:
import tensorflow as tf
from tensorflow import keras 
import os

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

In [12]:
keras.__version__
tf.__version__

'1.15.0'

In [13]:
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.6  # 0.6 sometimes works better for folks
#keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#if len(physical_devices) > 0:
#   tf.config.experimental.set_memory_growth(physical_devices[0], True)

#print(physical_devices)

In [14]:
def build(width, height, depth, classes):
# initialize the model along with the input shape to be
# "channels last"
    model = Sequential()
    inputShape = (height, width, depth)

    # if we are using "channels first", update the input shape
    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)

        # define the first (and only) CONV => RELU layer
    model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
    model.add(Activation("relu"))

    # softmax classifier
    model.add(Flatten())
    model.add(Dense(classes))
    model.add(Activation("softmax"))

        # return the constructed network architecture
    return model

In [15]:
trainX,trainY = cvpre.load_images('dataset/val_spl/')
testX,testY = cvpre.load_images('dataset/test_spl/')

In [16]:
trainX = cvpre.resize_ignore_aspect(trainX,200,200)
testX = cvpre.resize_ignore_aspect(testX,200,200)

In [17]:
trainX = cvpre.keras_dim_rearrange(trainX)
testX = cvpre.keras_dim_rearrange(testX)

In [18]:
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [19]:
# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

ValueError: y has 0 samples: array([], dtype=float64)

In [20]:
print("[INFO] compiling model...")
opt = SGD(lr=0.010)
model = build(width=200, height=200, depth=3, classes=196)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

[INFO] compiling model...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
trainX.shape

In [ ]:
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY),batch_size=32, epochs=100, verbose=1)